In [1]:
import networkx as nx
import copy
import gurobipy as gp
from gurobipy import GRB
from helpers import *
import time

# Instantiate processor

proc = Processor(4, 2, "HotspotConfiguration", 318.15)

# proc_cores is the number of processors per layer.
proc_layers = proc.nr_layers
proc_cores = proc.proc_per_layer

proc_total = proc.nr_pes
proc_config_path = proc.path_to_config


timestep = 1e-1

T_amb = proc.T_amb

coefficients = proc.coefficients


In [ ]:
def create_lb_model(graph : nx.DiGraph):
    """
    Creates and solves the makespan minimzation for task scheduling.

    Parameters:
    -----------
    - graph : nx.DiGraph
      A task graph. Each node should have the following attributes:
      - 'weight': The duration of the task.
      - 'power_cons': The power consumption of the task.

    Returns:
    --------
    - makespan : int
      The total time required to complete all tasks.
    - tasks_per_proc : list[list[tuple[int, float, float, float]]] 
      A list of lists, where each inner list contains tuples representing the tasks scheduled on each processor. Each tuple contains:
      - task number : int
      - start time : float
      - finish time : float
      - power consumption : float
    """
    nr_tasks = len(graph.nodes)
    nr_processors = proc_total
    task_lengths = [graph.nodes[i]['weight'] for i in graph.nodes]
    # task_lengths = [1 for i in graph.nodes]
    task_power = [graph.nodes[i]['power_cons'] for i in graph.nodes]
    
    # Create a new model
    m = gp.Model("Matching")

    # Create variables
    makespan = m.addVar(name="makespan")
    x = [[m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name = f"x[{i},{p}]") for p in range(nr_processors)] for i in range(nr_tasks)] # x[i,p] == 1 iff task i scheduled on PE p
    
    # Set objective: minimize makespan
    m.setObjective(1.0 * makespan, GRB.MINIMIZE)

    # Define makespan
    m.addConstrs(((sum(task_lengths[i] * x[i][p] for i in range(nr_tasks)) <= makespan) for p in range(nr_processors)), name="ms_def")
    
    # Task is planned on exactly one processor:
    m.addConstrs((sum(x[i][p] for p in range(nr_processors)) == 1 ) for i in range(nr_tasks))

    # Optimize model
    m.optimize()


    tasks_per_proc = [[] for _ in range(nr_processors)]

    load_per_p = [0 for p in range(nr_processors)]

    # Print the values of all variables
    for i in range(nr_tasks):
        for p in range(nr_processors):
            if x[i][p].X == 1.0:
                print("Task", i, "is mapped on PE", p, ". start time:", load_per_p[p], ", finish time:", load_per_p[p] + task_lengths[i])
                tasks_per_proc[p].append((i, load_per_p[p], load_per_p[p] + task_lengths[i], task_power[i]))
                load_per_p[p] += task_lengths[i]
    print(load_per_p)
    ms = round(makespan.X)
    m.dispose()
    return ms, tasks_per_proc



In [ ]:
def create_TATSND2_model(graph : nx.DiGraph):
    """
    Creates and solves the TATS-ND-2 problem.

    Parameters:
    -----------
    - graph : nx.DiGraph
      A task graph. Each node should have the following attributes:
      - 'weight': The duration of the task.
      - 'power_cons': The power consumption of the task.

    Returns:
    --------
    - makespan : int
      The total time required to complete all tasks.
    - tasks_per_proc : list[list[tuple[int, float, float, float]]] 
      A list of lists, where each inner list contains tuples representing the tasks scheduled on each processor. Each tuple contains:
      - task number : int
      - start time : float
      - finish time : float
      - power consumption : float
    """
    nr_tasks = len(graph.nodes)
    nr_processors = proc_total
    task_lengths = [graph.nodes[i]['weight'] for i in graph.nodes]
    # task_lengths = [1 for i in graph.nodes]
    task_power = [graph.nodes[i]['power_cons'] for i in graph.nodes]
    task_power_level = [graph.nodes[i]['power_level'] for i in graph.nodes]
    
    # Create a new model
    m = gp.Model("Matching")

    # Create variables
    makespan = m.addVar(name="makespan")
    start_time = [m.addVar(name=f"s({i})") for i in range(nr_tasks)]
    x = [[m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name = f"x[{i},{p}]") for p in range(nr_processors)] for i in range(nr_tasks)] # x[i,p] == 1 iff task i scheduled on PE p
    b = [[m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name=f"b[{i},{j}]") for j in range(nr_tasks)] for i in range(nr_tasks)] # b[i,j] == 1 iff task i finishes before task j starts
    o = [[m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name=f"o[{i},{j}]") for j in range(nr_tasks)] for i in range(nr_tasks)] # o[i,j] == 1 iff task i overlaps with task j

    # Set objective: minimize makespan
    m.setObjective(1.0 * makespan, GRB.MINIMIZE)

    # Define makespan
    m.addConstrs((start_time[i] + task_lengths[i] <= makespan for i in range(nr_tasks)), name="ms_def")

    # Task is planned on exactly one processor:
    m.addConstrs((sum(x[i][p] for p in range(nr_processors)) == 1 ) for i in range(nr_tasks))

    # Define b[i,j] and o[i,j]
    M = sum(task_lengths)  # Large constant: Worst case execution time, i.e. executing all tasks on the slowest core.
    m.addConstrs((start_time[i] + task_lengths[i] <= start_time[j] + M * (1 - b[i][j]) + M * o[i][j]) for i in range(nr_tasks) for j in range(nr_tasks))
    m.addConstrs((start_time[i] + M * (b[i][j] + o[i][j]) >= start_time[j] + task_lengths[j]) for i in range(nr_tasks) for j in range(nr_tasks))

    # If  two task on same processor, then one of them comes before the other
    m.addConstrs((x[i][p] + x[j][p] + o[i][j] <= 2) for i in range(nr_tasks) for j in range(nr_tasks) for p in range(nr_processors) if i != j)

    # No two tasks of power level 2 may overlap
    m.addConstrs((o[i][j] <= 0) for i in range(nr_tasks) for j in range(nr_tasks) if (i != j and task_power_level[i] == 1 and task_power_level[j] == 1))

    # Optimize model
    m.optimize()


    tasks_per_proc = [[] for _ in range(nr_processors)]

    # Print the values of all variables
    for i in range(nr_tasks):
        for p in range(nr_processors):
            if x[i][p].X == 1.0:
                print("Task", i, "is mapped on PE", p, ". start time:", start_time[i].X, ", finish time:", start_time[i].X + task_lengths[i])
                tasks_per_proc[p].append((i, start_time[i].X, start_time[i].X + task_lengths[i], task_power[i]))
    
    ms = round(makespan.X)
    m.dispose()
    return ms, tasks_per_proc



In [ ]:
def create_TATSND3_model(graph : nx.DiGraph):
    """
    Creates and solves the TATS-ND-3 problem.

    Parameters:
    -----------
    - graph : nx.DiGraph
      A task graph. Each node should have the following attributes:
      - 'weight': The duration of the task.
      - 'power_cons': The power consumption of the task.

    Returns:
    --------
    - makespan : int
      The total time required to complete all tasks.
    - tasks_per_proc : list[list[tuple[int, float, float, float]]] 
      A list of lists, where each inner list contains tuples representing the tasks scheduled on each processor. Each tuple contains:
      - task number : int
      - start time : float
      - finish time : float
      - power consumption : float
    """
    nr_tasks = len(graph.nodes)
    nr_processors = proc_total
    task_lengths = [graph.nodes[i]['weight'] for i in graph.nodes]
    # task_lengths = [1 for i in graph.nodes]
    task_power = [graph.nodes[i]['power_cons'] for i in graph.nodes]
    task_power_level = [graph.nodes[i]['power_level'] for i in graph.nodes]
    # task_power_level = [2 for i in graph.nodes]
    
    # Create a new model
    m = gp.Model("Matching")

    # Create variables
    makespan = m.addVar(name="makespan")
    start_time = [m.addVar(name=f"s({i})") for i in range(nr_tasks)]
    x = [[m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name = f"x[{i},{p}]") for p in range(nr_processors)] for i in range(nr_tasks)] # x[i,p] == 1 iff task i scheduled on PE p
    b = [[m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name=f"b[{i},{j}]") for j in range(nr_tasks)] for i in range(nr_tasks)] # b[i,j] == 1 iff task i finishes before task j starts
    o = [[m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name=f"o[{i},{j}]") for j in range(nr_tasks)] for i in range(nr_tasks)] # o[i,j] == 1 iff task i overlaps with task j

    # Set objective: minimize makespan
    m.setObjective(1.0 * makespan, GRB.MINIMIZE)

    # Define makespan
    m.addConstrs((start_time[i] + task_lengths[i] <= makespan for i in range(nr_tasks)), name="ms_def")

    # Task is planned on exactly one processor:
    m.addConstrs((sum(x[i][p] for p in range(nr_processors)) == 1 ) for i in range(nr_tasks))

    # Define b[i,j] and o[i,j]
    M = sum(task_lengths)  # Large constant: Worst case execution time, i.e. executing all tasks on the slowest core.
    m.addConstrs((start_time[i] + task_lengths[i] <= start_time[j] + M * (1 - b[i][j]) + M * o[i][j]) for i in range(nr_tasks) for j in range(nr_tasks))
    m.addConstrs((start_time[i] + M * (b[i][j] + o[i][j]) >= start_time[j] + task_lengths[j]) for i in range(nr_tasks) for j in range(nr_tasks))

    # If  two task on same processor, then one of them comes before the other
    m.addConstrs((x[i][p] + x[j][p] + o[i][j] <= 2) for i in range(nr_tasks) for j in range(nr_tasks) for p in range(nr_processors) if i != j)

    # No two tasks of power level 1 may overlap
    m.addConstrs((o[i][j] <= 0) for i in range(nr_tasks) for j in range(nr_tasks) if (i != j and task_power_level[i] == 1 and task_power_level[j] == 1))

    # No 3 tasks of power level 2 may overlap
    m.addConstrs((o[i][j] + o[j][k] + o[i][k] <= 2) for i in range(nr_tasks) for j in range(nr_tasks) for k in range(nr_tasks) if (i != j and j != k and i != k and task_power_level[i] == 2 and task_power_level[j] == 2 and task_power_level[k] == 2))

    # Optimize model
    m.optimize()


    tasks_per_proc = [[] for _ in range(nr_processors)]

    # Print the values of all variables
    for i in range(nr_tasks):
        for p in range(nr_processors):
            if x[i][p].X == 1.0:
                print("Task", i, "is mapped on PE", p, ". start time:", start_time[i].X, ", finish time:", start_time[i].X + task_lengths[i])
                tasks_per_proc[p].append((i, start_time[i].X, start_time[i].X + task_lengths[i], task_power[i]))
    
    ms = round(makespan.X)
    m.dispose()
    return ms, tasks_per_proc


In [ ]:
def create_TATSND4_model(graph : nx.DiGraph):
    """
    Creates and solves the TATS-ND-4 problem.

    Parameters:
    -----------
    - graph : nx.DiGraph
      A task graph. Each node should have the following attributes:
      - 'weight': The duration of the task.
      - 'power_cons': The power consumption of the task.

    Returns:
    --------
    - makespan : int
      The total time required to complete all tasks.
    - tasks_per_proc : list[list[tuple[int, float, float, float]]] 
      A list of lists, where each inner list contains tuples representing the tasks scheduled on each processor. Each tuple contains:
      - task number : int
      - start time : float
      - finish time : float
      - power consumption : float
    """
    nr_tasks = len(graph.nodes)
    nr_processors = proc_total
    task_lengths = [graph.nodes[i]['weight'] for i in graph.nodes]
    # task_lengths = [1 for i in graph.nodes]
    task_power = [graph.nodes[i]['power_cons'] for i in graph.nodes]
    task_power_level = [graph.nodes[i]['power_level'] for i in graph.nodes]
    # task_power_level = [2 for i in graph.nodes]
    
    # Create a new model
    m = gp.Model("Matching")

    # Create variables
    makespan = m.addVar(name="makespan")
    start_time = [m.addVar(name=f"s({i})") for i in range(nr_tasks)]
    x = [[m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name = f"x[{i},{p}]") for p in range(nr_processors)] for i in range(nr_tasks)] # x[i,p] == 1 iff task i scheduled on PE p
    b = [[m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name=f"b[{i},{j}]") for j in range(nr_tasks)] for i in range(nr_tasks)] # b[i,j] == 1 iff task i finishes before task j starts
    o = [[m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name=f"o[{i},{j}]") for j in range(nr_tasks)] for i in range(nr_tasks)] # o[i,j] == 1 iff task i overlaps with task j

    # Set objective: minimize makespan
    m.setObjective(1.0 * makespan, GRB.MINIMIZE)

    # Define makespan
    m.addConstrs((start_time[i] + task_lengths[i] <= makespan for i in range(nr_tasks)), name="ms_def")

    # Task is planned on exactly one processor:
    m.addConstrs((sum(x[i][p] for p in range(nr_processors)) == 1 ) for i in range(nr_tasks))

    # Define b[i,j] and o[i,j]
    M = sum(task_lengths)  # Large constant: Worst case execution time, i.e. executing all tasks on the slowest core.
    m.addConstrs((start_time[i] + task_lengths[i] <= start_time[j] + M * (1 - b[i][j]) + M * o[i][j]) for i in range(nr_tasks) for j in range(nr_tasks))
    m.addConstrs((start_time[i] + M * (b[i][j] + o[i][j]) >= start_time[j] + task_lengths[j]) for i in range(nr_tasks) for j in range(nr_tasks))

    # If  two task on same processor, then one of them comes before the other
    m.addConstrs((x[i][p] + x[j][p] + o[i][j] <= 2) for i in range(nr_tasks) for j in range(nr_tasks) for p in range(nr_processors) if i != j)

    # No two tasks of power level 1 may overlap
    m.addConstrs((o[i][j] <= 0) for i in range(nr_tasks) for j in range(nr_tasks) if (i != j and task_power_level[i] == 1 and task_power_level[j] == 1))

    # No 3 tasks of power level 2 may overlap
    m.addConstrs((o[i][j] + o[j][k] + o[i][k] <= 2) for i in range(nr_tasks) for j in range(nr_tasks) for k in range(nr_tasks) if (i != j and j != k and i != k and task_power_level[i] == 2 and task_power_level[j] == 2 and task_power_level[k] == 2))

    # No 4 tasks of power level 3 may overlap
    m.addConstrs((o[i][j] + o[i][k] + o[i][l] + o[j][k] + o[j][l] + o[k][l]  <= 5) for i in range(nr_tasks) for j in range(nr_tasks) for k in range(nr_tasks) for l in range(nr_tasks) if (len(set([i,j,k,l])) == 4 and task_power_level[i] == 3 and task_power_level[j] == 3 and task_power_level[k] == 3 and task_power_level[l] == 3))


    # Optimize model
    m.optimize()


    tasks_per_proc = [[] for _ in range(nr_processors)]

    # Print the values of all variables
    for i in range(nr_tasks):
        for p in range(nr_processors):
            if x[i][p].X == 1.0:
                print("Task", i, "is mapped on PE", p, ". start time:", start_time[i].X, ", finish time:", start_time[i].X + task_lengths[i])
                tasks_per_proc[p].append((i, start_time[i].X, start_time[i].X + task_lengths[i], task_power[i]))
    
    ms = round(makespan.X)
    m.dispose()
    return ms, tasks_per_proc



In [ ]:
def greedy_lb(graph : nx.DiGraph):
    """
    Finds a greedy solution to the Makespan Minimziation Problem.

    Parameters:
    -----------
    - graph : nx.DiGraph
      A task graph. Each node should have the following attributes:
      - 'weight': The duration of the task.
      - 'power_cons': The power consumption of the task.

    Returns:
    --------
    - makespan : int
      The total time required to complete all tasks.
    - tasks_per_proc : list[list[tuple[int, float, float, float]]] 
      A list of lists, where each inner list contains tuples representing the tasks scheduled on each processor. Each tuple contains:
      - task number : int
      - start time : float
      - finish time : float
      - power consumption : float
    """
    nr_tasks = len(graph.nodes)
    nr_processors = proc_total
    task_lengths = [graph.nodes[i]['weight'] for i in graph.nodes]
    # task_lengths = [1 for i in graph.nodes]
    task_power = [graph.nodes[i]['power_cons'] for i in graph.nodes]
    sorted_tasks = sorted([i for i in range(nr_tasks)], key=lambda x: task_lengths[x], reverse=True)

    load = [0] * nr_processors
    tasks_per_proc = [[] for _ in range(nr_processors)]

    for i in sorted_tasks:
        p = min(range(nr_processors), key = lambda x: load[x])
        tasks_per_proc[p].append((i, load[p], load[p] + task_lengths[i], task_power[i]))
        load[p] += task_lengths[i]
    
    return max(load), tasks_per_proc

def greedy_TATSND2(graph : nx.DiGraph):
    """
    Finds a greedy solution to the TATS-ND-2 Problem.

    Parameters:
    -----------
    - graph : nx.DiGraph
      A task graph. Each node should have the following attributes:
      - 'weight': The duration of the task.
      - 'power_cons': The power consumption of the task.

    Returns:
    --------
    - makespan : int
      The total time required to complete all tasks.
    - tasks_per_proc : list[list[tuple[int, float, float, float]]] 
      A list of lists, where each inner list contains tuples representing the tasks scheduled on each processor. Each tuple contains:
      - task number : int
      - start time : float
      - finish time : float
      - power consumption : float
    """
    nr_tasks = len(graph.nodes)
    nr_processors = proc_total
    task_lengths = [graph.nodes[i]['weight'] for i in graph.nodes]
    # task_lengths = [1 for i in graph.nodes]
    task_power = [graph.nodes[i]['power_cons'] for i in graph.nodes]
    task_power_level = [graph.nodes[i]['power_level'] for i in graph.nodes]

    load = [0] * nr_processors
    tasks_per_proc = [[] for _ in range(nr_processors)]
    sorted_tasks = sorted([i for i in range(nr_tasks)], key=lambda x: task_lengths[x], reverse=True)

    for i in sorted_tasks:
        p = 0
        if task_power_level[i] == 1:
            p = 7
        else:
            p = min(range(nr_processors - 1), key = lambda x: load[x])
        tasks_per_proc[p].append((i, load[p], load[p] + task_lengths[i], task_power[i]))
        load[p] += task_lengths[i]
    
    return max(load), tasks_per_proc

def greedy_TATSND3(graph : nx.DiGraph):
    """
    Finds a greedy solution to the TATS-ND-3 Problem.

    Parameters:
    -----------
    - graph : nx.DiGraph
      A task graph. Each node should have the following attributes:
      - 'weight': The duration of the task.
      - 'power_cons': The power consumption of the task.

    Returns:
    --------
    - makespan : int
      The total time required to complete all tasks.
    - tasks_per_proc : list[list[tuple[int, float, float, float]]] 
      A list of lists, where each inner list contains tuples representing the tasks scheduled on each processor. Each tuple contains:
      - task number : int
      - start time : float
      - finish time : float
      - power consumption : float
    """
    nr_tasks = len(graph.nodes)
    nr_processors = proc_total
    task_lengths = [graph.nodes[i]['weight'] for i in graph.nodes]
    task_power = [graph.nodes[i]['power_cons'] for i in graph.nodes]
    task_power_level = [graph.nodes[i]['power_level'] for i in graph.nodes]

    load = [0] * nr_processors
    tasks_per_proc = [[] for _ in range(nr_processors)]

    sorted_tasks = sorted([i for i in range(nr_tasks)], key=lambda x: task_lengths[x], reverse=True)
    for i in sorted_tasks:
        p = 0
        if task_power_level[i] == 1:
            p = 7
        elif task_power_level[i] == 2:
            p = min([4,6], key = lambda x: load[x])
        else:
            p = min([0,1,2,3,5], key = lambda x: load[x])
        tasks_per_proc[p].append((i, load[p], load[p] + task_lengths[i], task_power[i]))
        load[p] += task_lengths[i]
    
    return max(load), tasks_per_proc



In [ ]:

output_file_lb = "output_load_balancing.txt"
output_file_TATSND2 = "output_TATSND2.txt"
output_file_TATSND3 = "output_TATSND3.txt"
output_file_TATSND4 = "output_TATSND4.txt"

tpp1 = None
tpp2 = None
tpp3 = None
tpp4 = None
# Run Experiments for the (M)ILP formulations
for j in range(1, 41, 1):

    # Run experiments for Makespan Minimization Problem
    task_graph = create_random_graph(j)
    start = time.time()
    ms, tpp1 = create_lb_model(task_graph)
    proc.tpp_to_power_trace(tpp1, ms, output_file_lb, 1e-1)
    t_max, t_avg = proc.compute_max_ptrace_temp(output_file_lb, avg_temp=True)
    f = open(output_file_lb, 'a')
    f.write(f"Nr tasks {j}, Time {time.time() - start}, Makespan {ms}, Max Temp {t_max}, Avg Temp {t_avg} \n")
    f.close()


    # Run experiments for TATS-ND-2
    for i in range(len(task_graph.nodes)):
        task_graph.nodes[i]['power_level'] = 1 if task_graph.nodes[i]['power_cons'] >= 80 else 2
    start = time.time()
    ms, tpp2 = create_TATSND2_model(task_graph)
    proc.tpp_to_power_trace(tpp2, ms, output_file_TATSND2, 1e-1)
    t_max, t_avg = proc.compute_max_ptrace_temp(output_file_TATSND2, avg_temp=True)
    f = open(output_file_TATSND2, 'a')
    f.write(f"Nr tasks {j}, Time {time.time() - start}, Makespan {ms}, Max Temp {t_max}, Avg Temp {t_avg} \n")
    f.close()

    # Run experiments for TATS-ND-3
    for i in range(len(task_graph.nodes)):
        if task_graph.nodes[i]['power_cons'] >= 80:
            task_graph.nodes[i]['power_level'] = 1
        elif task_graph.nodes[i]['power_cons'] <= 45:
            task_graph.nodes[i]['power_level'] = 3
        else:
            task_graph.nodes[i]['power_level'] = 2

    start = time.time()
    ms, tpp3 = create_TATSND3_model(task_graph)
    proc.tpp_to_power_trace(tpp3, ms, output_file_TATSND3, 1e-1)
    end = time.time()
    t_max, t_avg = proc.compute_max_ptrace_temp(output_file_TATSND3, avg_temp=True)
    f = open(output_file_TATSND3, 'a')
    f.write(f"Nr tasks {j}, Time {end - start}, Makespan {ms}, Max Temp {t_max}, Avg Temp {t_avg} \n")
    f.close()


    # Run experiments for TATS-ND-4
    for i in range(len(task_graph.nodes)):
        if task_graph.nodes[i]['power_cons'] >= 80:
            task_graph.nodes[i]['power_level'] = 1
        elif task_graph.nodes[i]['power_cons'] <= 30:
            task_graph.nodes[i]['power_level'] = 4
        elif task_graph.nodes[i]['power_cons'] <= 45:
            task_graph.nodes[i]['power_level'] = 3
        else:
            task_graph.nodes[i]['power_level'] = 2

    start = time.time()
    ms, tpp4 = create_TATSND4_model(task_graph)
    proc.tpp_to_power_trace(tpp4, ms, output_file_TATSND4, 1e-1)
    end = time.time()
    t_max, t_avg = proc.compute_max_ptrace_temp(output_file_TATSND4, avg_temp=True)
    f = open(output_file_TATSND4, 'a')
    f.write(f"Nr tasks {j}, Time {end - start}, Makespan {ms}, Max Temp {t_max}, Avg Temp {t_avg} \n")
    f.close()


In [ ]:

output_file_lb = "output_load_balancing_approx_lowpower.txt"
output_file_TATSND2 = "output_TATSND2_approx_lowpower.txt"
output_file_TATSND3 = "output_TATSND3_approx_lowpower.txt"

tpp1 = None
tpp2 = None
tpp3 = None

# Run Experiments for the Greedy Formulations
for j in range(0, 1001, 10):
    # Run experiments for Makespan Minimization Problem
    task_graph = create_random_graph(j)
    start = time.time()
    ms, tpp1 = greedy_lb(task_graph)
    proc.tpp_to_power_trace(tpp1, ms, output_file_lb, 1e-1)
    t_max, t_avg = proc.compute_max_ptrace_temp(output_file_lb, avg_temp=True)
    f = open(output_file_lb, 'a')
    f.write(f"{j}; {time.time() - start}; {ms}; {t_max}; {t_avg} \n")
    f.close()

    # Run experiments for TATS-ND-2
    for i in range(len(task_graph.nodes)):
        task_graph.nodes[i]['power_level'] = 1 if task_graph.nodes[i]['power_cons'] >= 80 else 2
    start = time.time()
    ms, tpp2 = greedy_TATSND2(task_graph)
    proc.tpp_to_power_trace(tpp2, ms, output_file_TATSND2, 1e-1)
    t_max, t_avg = proc.compute_max_ptrace_temp(output_file_TATSND2, avg_temp=True)
    f = open(output_file_TATSND2, 'a')
    f.write(f"{j}; {time.time() - start}; {ms}; {t_max}; {t_avg} \n")
    f.close()

    # Run experiments for TATS-ND-3
    for i in range(len(task_graph.nodes)):
        if task_graph.nodes[i]['power_cons'] >= 80:
            task_graph.nodes[i]['power_level'] = 1
        elif task_graph.nodes[i]['power_cons'] <= 45:
            task_graph.nodes[i]['power_level'] = 3
        else:
            task_graph.nodes[i]['power_level'] = 2

    start = time.time()
    ms, tpp3 = greedy_TATSND3(task_graph)
    proc.tpp_to_power_trace(tpp3, ms, output_file_TATSND3, 1e-1)
    end = time.time()
    t_max, t_avg = proc.compute_max_ptrace_temp(output_file_TATSND3, avg_temp=True)
    f = open(output_file_TATSND3, 'a')
    f.write(f"{j}; {time.time() - start}; {ms}; {t_max}; {t_avg} \n")
    f.close()

